In [1]:
import os
import lightgbm as lgb
import pandas as pd
import warnings
import numpy as np
import random
import ast


# importo los .py
import sys
sys.path.append("auxiliares")
import config
import metricas
import lightgbm_aux
import extras

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
arch_entrada = "datasets/emp3_sellout_fe_estrellas.csv"
carpeta_base_exp = "exp/estrella/"

In [4]:
# Se genera la prediccion para el indice elegido de la optimizacion bayesiana
FUTURE_OPT_BAY_INDEX = 0
EXP_NOMBRE = "20231212_205536"

In [5]:
def lgbm_error_rate(preds, train_data):
    labels = train_data.get_label()
    ret_val = metricas.error_rate(labels,preds)
    return 'ER', ret_val, False

In [6]:
##############################################

In [7]:
carpeta_exp = carpeta_base_exp + EXP_NOMBRE + "/"

In [8]:
df_sellout = pd.read_csv(arch_entrada)
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  3900 non-null   int64  
 1   periodo                     3900 non-null   int64  
 2   customer_id                 3900 non-null   int64  
 3   cero_ventas                 3900 non-null   int64  
 4   plan_precios_cuidados       3900 non-null   float64
 5   cust_request_qty            3900 non-null   float64
 6   cust_request_tn             3900 non-null   float64
 7   tn                          3900 non-null   float64
 8   periodo_fecha               3900 non-null   object 
 9   mes                         3900 non-null   int64  
 10  meses_historia_prod         3900 non-null   int64  
 11  meses_historia_cliente      3900 non-null   int64  
 12  cliente_estrella            3900 non-null   int64  
 13  prod_cust                   3900 

In [9]:
### Categoricas
df_sellout = extras.convertir_categoricas_prod_cust(df_sellout)

df_sellout = df_sellout.drop(columns=["periodo_fecha","brand"])

In [10]:
# Modelo Final
cols_remover_entren = ["tn_mas_2","periodo"]

df_train_modelo_final = df_sellout[(df_sellout.periodo <= config.MODELO_FINAL_PERIODO_LIMITE_PARAM) & (df_sellout.periodo >= config.PERIODO_INICIO_PARAM)]
df_future = df_sellout[(df_sellout.periodo == config.FUTURE_PERIODO_PARAM)]

print("Periodos entrenar MODELO FINAL:",df_train_modelo_final.periodo.unique())
print("Periodos FUTURE:",df_future.periodo.unique())

# Variables entrenamiento
X_train_modelo_final = df_train_modelo_final.drop(columns=cols_remover_entren, axis=1)
y_train_modelo_final = df_train_modelo_final.tn_mas_2
cols_entren = X_train_modelo_final.columns.tolist()

X_future = df_future.drop(columns=cols_remover_entren, axis=1)

print("\nX_train_all:", X_train_modelo_final.shape)
print("y_train_all:", y_train_modelo_final.shape)
print("\nX_future:", X_future.shape)

lgbtrain_modelo_final = lgb.Dataset(data=X_train_modelo_final, label=y_train_modelo_final, feature_name=cols_entren)

Periodos entrenar MODELO FINAL: [201701 201702 201703 201704 201705 201706 201707 201708 201709 201710
 201711 201712 201801 201802 201803 201804 201805 201806 201807 201808
 201809 201810 201811 201812 201901 201902 201903 201904 201905 201906
 201907 201909 201910]
Periodos FUTURE: [201912]

X_train_all: (3660, 44)
y_train_all: (3660,)

X_future: (120, 44)


In [11]:
df_opt_bay = pd.read_excel(carpeta_exp + "opt_bay.xlsx")

future_params = ast.literal_eval(df_opt_bay.iloc[FUTURE_OPT_BAY_INDEX].lgb_params) #convierto string a diccionario
future_best_iter = int(df_opt_bay.iloc[FUTURE_OPT_BAY_INDEX].best_iteration)
y_pred_future = lightgbm_aux.semillerio(lgbtrain_modelo_final, future_params, future_best_iter, X_future,config.CANT_SEMILLAS)

In [12]:
# Guardo predicciones
y_pred_future = extras.limpiar_prediccion(y_pred_future)

# Prediccion Future
df_pred = pd.DataFrame(data={"product_id":X_future.product_id,"customer_id":X_future.customer_id,"tn":y_pred_future})
df_pred["product_customer_id"]="P"+df_pred["product_id"].astype(str) + '-C' + df_pred["customer_id"].astype(str)
df_pred = df_pred.drop(columns=["product_id","customer_id"])
df_pred.to_csv(carpeta_exp + str(FUTURE_OPT_BAY_INDEX) + "_prediccion.csv",index=False)